In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [2]:
import numpy as np 
import pandas as pd 
import gc
import os
import tensorflow as tf
import string
print(tf.__version__)

2.1.0-rc1


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
os.chdir('/content/drive/My Drive/Colab Notebooks Shared/IDA contests/Sentiment analysis')

Предобработка текстов не проводилась, так как сильного прироста в качестве не было, а без предобработки часть вычислений проходит в быстрее.

In [5]:
x_train = pd.read_csv('x_train.txt', sep='\r\t', header=None)
x_test = pd.read_csv('x_test.txt', sep='\r\t', header=None)
y_train = pd.read_csv('y_train.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [0]:
x_train = x_train.fillna(" ")
x_test = x_test.fillna(" ")

In [0]:
x_train1 = x_train.sample(n=3600000, random_state=32)
y_train1 = y_train.loc[x_train1.index]

Выберем эмбеддинги с меньшим количеством токенов и 100d векторами, для ускоренной обучаемости. Эксперименты показали, что большое количество токенов не очень хорошо для данного датасета.

In [14]:
!wget http://nlp.stanford.edu/data/wordvecs/glove.6B.zip

--2019-12-23 16:52:38--  http://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/wordvecs/glove.6B.zip [following]
--2019-12-23 16:52:39--  https://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip [following]
--2019-12-23 16:52:39--  http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182753 (822M) [applic

In [0]:
from tensorflow.keras.layers import Dense, Input, GRU, Embedding, Dropout, Bidirectional, Flatten, LeakyReLU, BatchNormalization, LSTM, SpatialDropout1D, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from tqdm import tqdm_notebook

In [0]:
import os, zipfile
file_name = os.path.abspath('./glove.6B.zip')
zip_ref = zipfile.ZipFile(file_name)
zip_ref.extractall('./')
zip_ref.close()
os.remove(file_name)

In [9]:
f = open('./glove.6B.100d.txt')
embeddings_index = dict() 
for line in tqdm_notebook(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [0]:
x_train1.columns = ['text']
x_test.columns = ['text']

In [0]:
x_train1['text'] = x_train1['text'].apply(lambda x: str(x))
x_test['text'] = x_test['text'].apply(lambda x: str(x))

In [0]:
embed_size = 100 
max_features = 20000 
maxlen = 150 # медианная длина текстов варьируется около этого числа

In [0]:
list_sentences_train = x_train1.iloc[:,0].values
y = y_train1['Probability'].values
list_sentences_test = x_test.iloc[:,0].values

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
x_train_pad = pad_sequences(list_tokenized_train, maxlen=maxlen)
x_test_pad = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [14]:
all_embs = np.stack(embeddings_index.values())
emb_mean, emb_std = all_embs.mean(), all_embs.std()
emb_mean, emb_std

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


(0.0044520576, 0.40815717)

In [0]:
word_index = tokenizer.word_index
unknown_words = set()
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    else: unknown_words.add(word)

Используем Коллбеки, которые позволят каждую эпоху не только сохранять модель, но и предсказывать и сохранять результаты для test датасета.

In [0]:
from tensorflow.keras.models import save_model, load_model

class ModelEndCallback(tf.keras.callbacks.Callback):

    def __init__(self, file_name, csv_name):
        super(ModelEndCallback, self).__init__()
        self.file_name = file_name
        self.csv_name = csv_name

    def on_epoch_end(self, epoch, logs=None):
        save_model(self.model, self.file_name.format(epoch+1))
        print('\n', self.file_name.format(epoch+1),)

        res = self.model.predict(x_test_pad, batch_size=None, verbose=0)
        df = pd.DataFrame({'Id': range(1, len(res)+1), 'Probability': res.T[0]})
        df.to_csv(self.csv_name.format(epoch+1), index=False)

def load_from_file(model_filename, last_epoch):
    return load_model(model_filename.format(last_epoch))

В данной модели используются обучающиеся эмбеддинги, слой LSTM с дропаутом для защиты от переобучения. Далее соединяем два слоя MaxPooling и AveragePooling. Используем полносвязный слой на 64. Слишком большие слои не дают значительного прироста результата и обучаются долго.

In [0]:
model_filename = 'final_weights_{0:02d}.hdf5'
model_csvname = 'final_results_ep{0:02d}.csv'

initializer = tf.keras.initializers.glorot_normal()
input_layer = Input((maxlen,), name = '0')
embedding_layer = Embedding(max_features, embed_size, input_length=maxlen, 
                            weights=[embedding_matrix], 
                            trainable = True)(input_layer)
x = Bidirectional(LSTM(64, return_sequences=True, dropout=0.1, 
                       recurrent_dropout=0.1))(embedding_layer)

x1 = GlobalMaxPooling1D()(x)
x2 = GlobalAveragePooling1D()(x)

x = concatenate([x1, x2])
x = Dense(64, activation="relu")(x)
x = Dropout(0.1)(x)
output_layer = Dense(1, activation="sigmoid")(x)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='binary_crossentropy',
                  optimizer=Adam(clipnorm=2, clipvalue=2),
                  metrics=['AUC'])

def schedule(ind):
    a = [0.001, 0.001, 0.0001, 0.0001, 0.00001, 0.00001, 0.000001]
    return a[ind]

lr = LearningRateScheduler(schedule)
    
early_stop = EarlyStopping(monitor='val_loss',
                           patience=4,
                           verbose=1,
                           min_delta=1e-4)

In [18]:
history = model.fit(x_train_pad, y, batch_size=512, epochs = 2, 
                    validation_split = 0.1, verbose = 1, callbacks = [lr, ModelEndCallback(model_filename, model_csvname)])

Train on 3240000 samples, validate on 360000 samples
Epoch 1/2
3239936/3240000 [============================>.] - ETA: 0s - loss: 0.1723 - AUC: 0.9814
 final_weights_01.hdf5
3240000/3240000 [==============================] - 3638s 1ms/sample - loss: 0.1723 - AUC: 0.9814 - val_loss: 0.1385 - val_AUC: 0.9876
Epoch 2/2
3239936/3240000 [============================>.] - ETA: 0s - loss: 0.1324 - AUC: 0.9884
 final_weights_02.hdf5
3240000/3240000 [==============================] - 3667s 1ms/sample - loss: 0.1324 - AUC: 0.9884 - val_loss: 0.1294 - val_AUC: 0.9889
